In [1]:
# ensure classes imported from .py files are dynamically updated
%load_ext autoreload
%autoreload 2

# plot matplots nicely
%matplotlib inline  

In [2]:
import numpy as np
import pandas as pd
import datetime
import xarray as xr
from dateutil.relativedelta import relativedelta
from climate_drought import config, drought_indices as dri, utils

In [24]:
# configure inputs
#aa = config.AnalysisArgs(latitude=36,longitude=-120,start_date='20200116',end_date='20200410')
aa = config.AnalysisArgs(latitude=52.5,longitude=1.25,start_date='20200216',end_date='20221231')
cf = config.Config(outdir= 'output',indir='input',verbose=False)

In [34]:
cdi = dri.CDI(cf,aa)
cdi.download()
cdi.process()

2023-04-17  12:57:59 INFO Downloaded file 'output\precip_19850101-20221231_monthly_52.5_1.25.nc' already exists.
2023-04-17  12:57:59 INFO Downloaded files available.
2023-04-17  12:57:59 INFO Downloaded files available.
2023-04-17  12:57:59 INFO Computing Combined Drought Indicator...
2023-04-17  12:57:59 INFO Initiating processing of ERA5 daily data.
2023-04-17  12:57:59 INFO Input precipitation, 456 values: 0.000 0.005 
2023-04-17  12:57:59 INFO 

2023-04-17  12:57:59 DEBUG scaled values: 0.002 0.012
2023-04-17  12:57:59 DEBUG alphas: 9.687 17.504 betas: 0.000 0.001
2023-04-17  12:57:59 INFO SPI, 456 values: -3.046 2.676
2023-04-17  12:58:01 INFO Completed processing of ERA5 soil water data.
2023-04-17  12:58:02 INFO Completed processing of ERA5 fAPAR data.
2023-04-17  12:58:02 INFO Completed processing of ERA5 CDI data.


,tp,spi,smant,fpanv,CDI
2020-01-01,NaN,NaN,NaN,NaN,NaN
2020-01-11,NaN,NaN,NaN,NaN,NaN
2020-01-21,NaN,NaN,NaN,NaN,NaN
2020-02-01,0.001882,0.928141,NaN,NaN,NaN
2020-02-11,0.001882,0.928141,-0.08,-1.078,0.0
...,...,...,...,...,...
2022-11-11,0.000909,-1.089360,-1.81,1.780,0.0
2022-11-21,0.000909,-1.089360,-0.40,NaN,NaN
2022-12-01,0.002387,-0.953641,NaN,NaN,NaN
2022-12-11,0.002387,-0.953641,NaN,NaN,NaN


In [35]:
warns = []
for i,r in cdi.df_shifted.iterrows():
    spi_warn = r['spi'] < -1
    warns.append(r['spi'] < -1)

In [48]:
cdi.df_shifted.isna().any(axis=1)

2020-01-01     True
2020-01-11     True
2020-01-21     True
2020-02-01     True
2020-02-11    False
              ...  
2022-11-11    False
2022-11-21     True
2022-12-01     True
2022-12-11     True
2022-12-21     True
Length: 108, dtype: bool

In [52]:

def calc_cdi(r):
    spi_ = r['spi'] < -1
    sma_ = r['smant'] < -1
    fpr_ = r['fpanv'] < -1
    if r.isna().any():
        return np.nan
    elif spi_ and sma_ and fpr_:
        return 4
    elif spi_ and fpr_:
        return 3
    elif spi_ and sma_:
        return 2
    elif spi_:
        return 1
    else:
        return 0
    
cdi.df_shifted.apply(calc_cdi,axis=1)

NameError: name 'spi' is not defined

In [14]:
(cdi.sma_shifted < -1).to_numpy().flatten()

array([False, False, False, False, False, False, False,  True,  True,
        True, False, False,  True,  True,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
        True, False, False, False, False])